# Models API

使用 Models API 查看和访问 OpenAI 提供的预训练大语言模型

## List Models

列出DeepSeek当前可用的模型，并提供每个模型的基本信息，如所有者和可用性。

In [1]:
from openai import OpenAI

# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
client = OpenAI(api_key="<替换为api-key>", base_url="https://api.deepseek.com")
models = client.models.list()

In [2]:
print(models)

SyncPage[Model](data=[Model(id='deepseek-chat', created=None, object='model', owned_by='deepseek'), Model(id='deepseek-reasoner', created=None, object='model', owned_by='deepseek')], object='list')


#### 查看 OpenAI 最新提供的模型 API 信息

`models.data`: 目前OpenAI提供的大语言模型列表，列表中的每一项都对应着一个模型实例。

以`deepseek-chat`模型为例，解释说明各项参数：

1. `created`: 这是模型创建的时间戳，单位为 Unix 时间戳（自1970年1月1日（00:00:00 GMT）以后的秒数），值为 None 可能表示时间未记录或未返回。。
2. `id`: ‌模型唯一标识符‌，用于区分不同模型（如 API 调用时指定模型）。在这个例子中，模型的 ID 是 "deepseek-chat"。
3. `object`: 这个字段表示的是当前对象的类型，在这个例子中，对象是 "model"，固定标识符，表示这是一个“模型”类型的资源对象。
4. `owned_by`: 模型所有者/归属方，通常是组织名（如公司、团队或开发者名称）。在这个例子中，模型的所有者是 "deepseek"。

In [3]:
models.data

[Model(id='deepseek-chat', created=None, object='model', owned_by='deepseek'),
 Model(id='deepseek-reasoner', created=None, object='model', owned_by='deepseek')]

### 获取模型 ID 列表

In [4]:
models.data[0].id

'deepseek-chat'

In [5]:
model_list = [model.id for model in models.data]

In [6]:
print(model_list)

['deepseek-chat', 'deepseek-reasoner']


## Retrieve Model

根据前面查询到当前支持的模型ID列表，获取指定模型实例，如`deepseek-chat`。

In [9]:
# 将模型 ID 传入 retrieve 接口
deepseek_chat_v3 = client.models.retrieve("deepseek-chat")

In [10]:
print(deepseek_chat_v3)

Model(id='deepseek-chat', created=None, object='model', owned_by='deepseek')


### 获取指定模型，如 deepseek-reasoner

In [11]:
client.models.retrieve("deepseek-reasoner")

Model(id='deepseek-reasoner', created=None, object='model', owned_by='deepseek')

# 文本内容补全初探（Completions API）[Legacy]

使用 Completions API 实现各类文本生成任务


主要请求参数说明：


- **`model`** （string，必填）

  要使用的模型的 ID。可以参考 **模型端点兼容性表**。

- **`prompt`** （string or array，必填，Defaults to ）

  生成补全的提示，编码为字符串、字符串数组、token数组或token数组数组。

  注意，这是模型在训练过程中看到的文档分隔符，所以如果没有指定提示符，模型将像从新文档的开头一样生成。

- **`stream`** （boolean，选填，默认 false）

  当它设置为 true 时，API 会以 SSE（ Server Side Event ）方式返回内容，即会不断地输出内容直到完成响应，流通过 `data: [DONE]` 消息终止。

- **`max_tokens`** （integer，选填，默认是 16）

  补全时要生成的最大 token 数。

  提示 `max_tokens` 的 token 计数不能超过模型的上下文长度。大多数模型的上下文长度为 2048 个token（最新模型除外，它支持 4096）

- **`temperature`** （number，选填，默认是1）

  使用哪个采样温度，在 **0和2之间**。

  较高的值，如0.8会使输出更随机，而较低的值，如0.2会使其更加集中和确定性。

  通常建议修改这个（`temperature` ）或 `top_p` 但两者不能同时存在，二选一。

- **`n`** （integer，选填，默认为 1）

  每个 `prompt` 生成的补全次数。

  注意：由于此参数会生成许多补全，因此它会快速消耗token配额。小心使用，并确保对 `max_tokens` 和 `stop` 进行合理的设置。


## 生成英文文本

In [14]:
# 上面这种是openai的接口文档，直接用prompt
# data = client.completions.create(
#   model="deepseek-chat",
#   prompt="Say this is a test",
#   max_tokens=7,
#   temperature=0
# )
# deepseek需要用messages
data = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
  ],
    max_tokens=1024,
    temperature=0.7,
    stream=False
)


In [15]:
print(data)

ChatCompletion(id='7b247840-1f64-476f-a5a3-e80fd63cdc98', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today? 😊', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1740710775, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_3a5770e1b4_prod0225', usage=CompletionUsage(completion_tokens=11, prompt_tokens=9, total_tokens=20, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=9))


In [17]:
text = data.choices[0].message.content

In [18]:
print(text)

Hello! How can I assist you today? 😊


In [26]:
from openai import OpenAI

# user should set `base_url="https://api.deepseek.com/beta"` to use this feature.
client1 = OpenAI(
  api_key="<替换为api-key>",
  base_url="https://api.deepseek.com/beta",
)
response = client1.completions.create(
  model="deepseek-chat",
  prompt="def fib(a):",
  suffix="    return fib(a-1) + fib(a-2)",
  max_tokens=128)
print(response.choices[0].text)


    if a == 0:
        return 0
    elif a == 1:
        return 1
    else:
    


In [27]:
print(response)

Completion(id='9e34567d-880b-4501-9ad0-268d46373bdb', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n    if a == 0:\n        return 0\n    elif a == 1:\n        return 1\n    else:\n    ')], created=1740712319, model='deepseek-chat', object='text_completion', system_fingerprint='fp_3a5770e1b4_prod0225', usage=CompletionUsage(completion_tokens=29, prompt_tokens=21, total_tokens=50, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=21))


## 生成中文文本

调整 `max_tokens` 

In [21]:
# deepseek需要用messages
data = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "讲2个给程序员听的笑话"},
  ],
    max_tokens=1024,
    temperature=1.2,
    stream=False
)

In [22]:
text = data.choices[0].message.content
print(text)

当然！以下是两个程序员可能会喜欢的笑话：

---

**笑话1：**  
程序员A：我昨天写了一个无限循环的代码。  
程序员B：然后呢？  
程序员A：我把它删了，因为我不想让电脑崩溃。  
程序员B：那你为什么不加个 `break` 语句？  
程序员A：因为我想让代码保持“纯粹”。  

---

**笑话2：**  
面试官：你能解释一下递归吗？  
程序员：当然，递归就是“你能解释一下递归吗？”  

---

希望你喜欢！程序员的笑话总是充满了代码和逻辑的幽默感 😄


## 生成 Python 代码，并执行和验证

以面试中考察的典型的试题 `快速排序` 为例

In [23]:
# deepseek需要用messages
data = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "生成可执行的快速排序 Python 代码"},
  ],
    max_tokens=1024,
    temperature=1.2,
    stream=False
)

In [24]:
text = data.choices[0].message.content
print(text)

当然！以下是一个使用快速排序算法实现的 Python 代码：

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

# 示例用法
arr = [3, 6, 8, 10, 1, 2, 1]
sorted_arr = quicksort(arr)
print("排序后的数组:", sorted_arr)
```

### 代码说明：
1. **quicksort 函数**：这是一个递归函数，用于对数组进行快速排序。
   - 如果数组的长度小于等于 1，直接返回数组（递归的基准条件）。
   - 选择一个基准值（pivot），通常选择数组中间的元素。
   - 将数组分为三部分：小于基准值的元素、等于基准值的元素和大于基准值的元素。
   - 递归地对小于和大于基准值的部分进行排序，然后将结果合并。

2. **示例用法**：
   - 定义一个未排序的数组 `arr`。
   - 调用 `quicksort` 函数对数组进行排序。
   - 打印排序后的数组。

### 输出：
```python
排序后的数组: [1, 1, 2, 3, 6, 8, 10]
```

这个实现是快速排序的一个简单版本，适用于理解算法的工作原理。在实际应用中，可能需要考虑更多的优化和边界条件处理。


In [25]:
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)

# 示例用法
arr = [3, 6, 8, 10, 1, 2, 1]
sorted_arr = quicksort(arr)
print("排序后的数组:", sorted_arr)

排序后的数组: [1, 1, 2, 3, 6, 8, 10]


# 聊天机器人初探（Chat Completions API）

使用 Chat Completions API 实现对话任务

聊天补全(Chat Completions API)以消息列表作为输入，并返回模型生成的消息作为输出。尽管聊天格式旨在使多轮对话变得简单，但它同样适用于没有任何对话的单轮任务。

主要请求参数说明：


- **`model` （string，必填）**

  要使用的模型ID。有关哪些模型适用于Chat API的详细信息

- **`messages` （array，必填）**

  迄今为止描述对话的消息列表
    - **`role` （string，必填）**

  发送此消息的角色。`system` 、`user` 或 `assistant` 之一（一般用 user 发送用户问题，system 发送给模型提示信息）

    - **`content` （string，必填）**
    
      消息的内容
    
    - **`name` （string，选填）**
    
      此消息的发送者姓名。可以包含 a-z、A-Z、0-9 和下划线，最大长度为 64 个字符

- **`stream` （boolean，选填，是否按流的方式发送内容）**

  当它设置为 true 时，API 会以 SSE（ Server Side Event ）方式返回内容。SSE 本质上是一个长链接，会持续不断地输出内容直到完成响应。如果不是做实时聊天，默认false即可。

- **`max_tokens` （integer，选填）**

  在聊天补全中生成的最大 **tokens** 数。

  输入token和生成的token的总长度受模型上下文长度的限制。

- **`temperature` （number，选填，默认是 1）**

  采样温度，在 0和 2 之间。

  较高的值，如0.8会使输出更随机，而较低的值，如0.2会使其更加集中和确定性。

  通常建议修改这个（`temperature` ）或者 `top_p` ，但两者不能同时存在，二选一。


## 开启聊天模式

使用 `messages` 记录迄今为止对话的消息列表

In [28]:
messages=[
    {
        "role": "user", 
        "content": "Hello!"
    }
]


data = client.chat.completions.create(
  model="deepseek-chat",
  messages = messages
)


In [29]:
print(data)

ChatCompletion(id='07f2cce7-0e5c-474c-acfa-5aa2b674a41e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today? 😊', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1740712564, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_3a5770e1b4_prod0225', usage=CompletionUsage(completion_tokens=11, prompt_tokens=5, total_tokens=16, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=5))


In [30]:
# 从返回的数据中获取生成的消息
new_message = data.choices[0].message
# 打印 new_message
print(new_message)

ChatCompletionMessage(content='Hello! How can I assist you today? 😊', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [31]:
# 将消息追加到 messages 列表中
messages.append(new_message)
print(messages)

[{'role': 'user', 'content': 'Hello!'}, ChatCompletionMessage(content='Hello! How can I assist you today? 😊', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)]


In [32]:
type(new_message)

openai.types.chat.chat_completion_message.ChatCompletionMessage

In [33]:
new_message.role

'assistant'

In [34]:
new_message.content

'Hello! How can I assist you today? 😊'

In [35]:
messages.pop()

ChatCompletionMessage(content='Hello! How can I assist you today? 😊', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [36]:
print(messages)

[{'role': 'user', 'content': 'Hello!'}]


#### Prompt: OpenAIObject -> Dict

```
打印 messages 列表后发现数据类型不对，messages 输出如下：

print(messages)

[{'role': 'user', 'content': 'Hello!'}, <OpenAIObject at 0x7f27582c13f0> JSON: {
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}]

将OpenAIObject 转换为一个如下数据类型格式：

    {
        "role": "user", 
        "content": "Hello!"
    }
```

In [37]:
new_message = data.choices[0].message
new_message_dict = {"role": new_message.role, "content": new_message.content}
type(new_message_dict)

dict

In [38]:
print(new_message_dict)

{'role': 'assistant', 'content': 'Hello! How can I assist you today? 😊'}


In [43]:
# 将消息追加到 messages 列表中
messages.append(new_message_dict)

In [44]:
print(messages)

[{'role': 'user', 'content': 'Hello!'}, {'role': 'user', 'content': '1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today? 😊'}]


#### 新一轮对话

In [39]:
new_chat = {
    "role": "user",
    "content": "1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？"
}

In [40]:
messages.append(new_chat)

In [45]:
from pprint import pprint

pprint(messages)

[{'content': 'Hello!', 'role': 'user'},
 {'content': '1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？', 'role': 'user'},
 {'content': 'Hello! How can I assist you today? 😊', 'role': 'assistant'}]


In [46]:
data = client.chat.completions.create(
  model="deepseek-chat",
  messages=messages
)

In [47]:
new_message = data.choices[0].message
# 打印 new_messages 
print(new_message)

ChatCompletionMessage(content='1. 讲一个程序员才听得懂的冷笑话：  \n为什么程序员总是分不清万圣节和圣诞节？  \n因为 Oct 31 = Dec 25。  \n（解释：八进制的31等于十进制的25。）\n\n2. 今天是几号？  \n今天是2023年10月5日。\n\n3. 明天星期几？  \n明天是2023年10月6日，星期五。\n\n希望你喜欢这个冷笑话！😄', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [48]:
# 打印 new_messages 内容
print(new_message.content)

1. 讲一个程序员才听得懂的冷笑话：  
为什么程序员总是分不清万圣节和圣诞节？  
因为 Oct 31 = Dec 25。  
（解释：八进制的31等于十进制的25。）

2. 今天是几号？  
今天是2023年10月5日。

3. 明天星期几？  
明天是2023年10月6日，星期五。

希望你喜欢这个冷笑话！😄


## 使用多种身份聊天对话

目前`role`参数支持3类身份： `system`, `user` `assistant`:


![](images/chat_completion_api.png)



In [3]:
# 构造聊天记录
messages=[
    {"role": "system", "content": "你是一个乐于助人的体育界专家。"},
    {"role": "user", "content": "2008年奥运会是在哪里举行的？"},
]

In [4]:
data = client.chat.completions.create(
  model="deepseek-chat",
  messages=messages
)


In [5]:
message = data.choices[0].message.content
print(message)

2008年奥运会是在**北京**举行的。这届奥运会是第29届夏季奥林匹克运动会，于2008年8月8日至8月24日在北京举行。这是中国首次举办夏季奥运会，也是继1964年东京奥运会和1988年汉城奥运会后，第三个举办夏季奥运会的亚洲城市。北京奥运会的主题口号是“同一个世界，同一个梦想”，体现了奥林匹克精神的核心价值。


In [6]:
# 添加 deep-seek 返回结果到聊天记录
messages.append({"role": "assistant", "content": message})

In [7]:
messages

[{'role': 'system', 'content': '你是一个乐于助人的体育界专家。'},
 {'role': 'user', 'content': '2008年奥运会是在哪里举行的？'},
 {'role': 'assistant',
  'content': '2008年奥运会是在**北京**举行的。这届奥运会是第29届夏季奥林匹克运动会，于2008年8月8日至8月24日在北京举行。这是中国首次举办夏季奥运会，也是继1964年东京奥运会和1988年汉城奥运会后，第三个举办夏季奥运会的亚洲城市。北京奥运会的主题口号是“同一个世界，同一个梦想”，体现了奥林匹克精神的核心价值。'}]

In [8]:
# 第二轮对话
messages.append({"role": "user", "content": "1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？"})

In [10]:
messages

[{'role': 'system', 'content': '你是一个乐于助人的体育界专家。'},
 {'role': 'user', 'content': '2008年奥运会是在哪里举行的？'},
 {'role': 'assistant',
  'content': '2008年奥运会是在**北京**举行的。这届奥运会是第29届夏季奥林匹克运动会，于2008年8月8日至8月24日在北京举行。这是中国首次举办夏季奥运会，也是继1964年东京奥运会和1988年汉城奥运会后，第三个举办夏季奥运会的亚洲城市。北京奥运会的主题口号是“同一个世界，同一个梦想”，体现了奥林匹克精神的核心价值。'},
 {'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]

In [ ]:
data = client.chat.completions.create(
  model="deepseek-chat",
  messages=messages
)

In [58]:
message = data.choices[0].message.content
print(message)

在2008年北京奥运会上：

1. **金牌最多的是中国**。中国代表团在本届奥运会上表现出色，共获得了51枚金牌，这是中国在奥运会历史上获得金牌最多的一次，也使得中国首次在奥运会金牌榜上位居第一。

2. **奖牌总数最多的是美国**。美国代表团在2008年奥运会上共获得了110枚奖牌，其中包括36枚金牌、38枚银牌和36枚铜牌。尽管在金牌数上略低于中国，但美国在奖牌总数上仍然领先。

这两项成就都反映了中国和美国在体育领域的强大实力和竞争力。


In [48]:
data = client.chat.completions.create(
  model="deepseek-chat",
  messages=[{'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]
)

In [49]:
data.choices[0].message.content

'1. 根据2021年东京奥运会的数据，金牌最多的国家是美国。\n\n2. 根据2021年东京奥运会的数据，奖牌最多的国家是美国。'